In [0]:
%sql
/*********************************************
SELECT
  COUNT(*) AS total_rows,
  COUNT(CASE WHEN apoio_concessionario IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_concessionario,
  COUNT(CASE WHEN apoio_total IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_total,
  COUNT(CASE WHEN sub_total_com_extras IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_sub_total_com_extras,
  COUNT(CASE WHEN descricao_do_pedido_de_apoio IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_do_pedido_de_apoio,
  COUNT(CASE WHEN estado_do_pedido IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_pedido,
  COUNT(CASE WHEN apoio_percentual IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_percentual,
  COUNT(CASE WHEN data_de_criacao_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_criacao_da_proposta,
  COUNT(CASE WHEN valor_campanhas_comerciais IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_campanhas_comerciais,
  COUNT(CASE WHEN ofertas_de_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_ofertas_de_campanha,
  COUNT(CASE WHEN resposta_do_importador IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_resposta_do_importador,
  COUNT(CASE WHEN data_de_entrega_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_entrega_da_proposta,
  COUNT(CASE WHEN desconto_total__c__apoio_de_importador_ IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_desconto_total__c__apoio_de_importador_,
  COUNT(CASE WHEN codigo_cor_exterior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_exterior,
  COUNT(CASE WHEN codigo_cor_interior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_interior,
  COUNT(CASE WHEN concessionario_owner IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_concessionario_owner,
  COUNT(CASE WHEN valor_aprovado IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_aprovado,
  COUNT(CASE WHEN estado_do_contrato IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_contrato,
  COUNT(CASE WHEN data_prevista_matricula IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_matricula,
  COUNT(CASE WHEN data_prevista_de_entrega IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_de_entrega,
  COUNT(CASE WHEN data_da_conclusao IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_da_conclusao,
  COUNT(CASE WHEN id_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_classe,
  COUNT(CASE WHEN descricao_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_classe,
  COUNT(CASE WHEN id_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_model_group,
  COUNT(CASE WHEN descricao_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_model_group,
  COUNT(CASE WHEN valid_until IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valid_until,
  COUNT(CASE WHEN forma_de_pagamento IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_forma_de_pagamento,
  COUNT(CASE WHEN nome_da_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_nome_da_campanha,
  COUNT(CASE WHEN _fivetran_index IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_fivetran_index
FROM workspace.sc_silver.propostas_realizadas;
*********************************************/

In [0]:
%sql
/*********************************************
SELECT 
  COUNT(DISTINCT c.id) AS common_unique_ids,
  COUNT(DISTINCT CASE 
    WHEN c.data_criacao_da_lead = l.created_time THEN c.id
  END) AS common_unique_ids_with_date
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;
*********************************************/

In [0]:
%sql
/*********************************************
SELECT `n__matriculas_por_associar`, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY 1
ORDER BY cnt DESC;

-- sanity check there are no negatives left
SELECT COUNT(*) AS negatives_left
FROM sc_silver.deals
WHERE COALESCE(
        try_cast(`n__matriculas_por_associar` AS BIGINT),
        try_cast(regexp_replace(trim(CAST(coalesce(`n__matriculas_por_associar`,'') AS STRING)),
                                '[^0-9-]', '') AS BIGINT)
      ) < 0;
*********************************************/

In [0]:
%sql
SELECT * FROM sc_silver.viaturas_demo;

In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas_demo"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # Compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)

In [0]:
%sql

SELECT * FROM sc_silver.viaturas_demo;

In [0]:
%sql
-- sc_silver.viaturas_demo
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.viaturas_demo t
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas_demo
GROUP BY 1
ORDER BY cnt DESC;

In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas_demo
GROUP BY 1
ORDER BY cnt DESC;

In [0]:
%sql
SELECT * FROM sc_silver.contas_pbs;

In [0]:
%sql
-- sc_silver.contas_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contas_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.contas_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campaigns;

In [0]:
%sql
SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
%sql
SELECT * FROM sc_silver.deals;

In [0]:
%sql
SHOW TABLES IN workspace.sc_silver;

In [0]:
%sql
SELECT * FROM sc_silver.historico_de_servicos;

In [0]:
%sql
SELECT * FROM sc_silver.leads_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas;